In [1]:
import os
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
n_gpu = torch.cuda.device_count()

/home/m13519061/anaconda3/envs/absa/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

In [3]:
import sys
from transformers import AutoModelForCausalLM, AutoTokenizer
sys.path.append("../../../src/")
import data_utils

# Dataset Utilities

In [4]:
peng_dir = dict(
    lap14 = "../../../data/absa/en/peng/14lap",
    res14 = "../../../data/absa/en/peng/14res",
    res15 = "../../../data/absa/en/peng/15res",
    res16 = "../../../data/absa/en/peng/16res"
)

wan_dir = dict(
    res15 = "../../../data/absa/en/wan/interim/rest15",
    res16 = "../../../data/absa/en/wan/interim/rest16"
)
    
zhang_dir = dict(
    res15 = "../../../data/absa/en/zhang/interim/interim_2/rest15",
    res16 = "../../../data/absa/en/zhang/interim/interim_2/rest16"
)

william_dir = dict(
    hotel = "../../../data/absa/id/william"
)

peng = dict(
    lap14 = dict(
        train = data_utils.read_data(path=peng_dir["lap14"] + "/train_triplets.txt",
                                     target_format="aos"),
        val = data_utils.read_data(path=peng_dir["lap14"] + "/dev_triplets.txt",
                                     target_format="aos"),
        test = data_utils.read_data(path=peng_dir["lap14"] + "/test_triplets.txt",
                                     target_format="aos")
    ),
    res14 = dict(
        train = data_utils.read_data(path=peng_dir["res14"] + "/train_triplets.txt",
                                     target_format="aos"),
        val = data_utils.read_data(path=peng_dir["res14"] + "/dev_triplets.txt",
                                     target_format="aos"),
        test = data_utils.read_data(path=peng_dir["res14"] + "/test_triplets.txt",
                                     target_format="aos")
    ),
    res15 = dict(
        train = data_utils.read_data(path=peng_dir["res15"] + "/train_triplets.txt",
                                     target_format="aos"),
        val = data_utils.read_data(path=peng_dir["res15"] + "/dev_triplets.txt",
                                     target_format="aos"),
        test = data_utils.read_data(path=peng_dir["res15"] + "/test_triplets.txt",
                                     target_format="aos")
    ),
    res16 = dict(
        train = data_utils.read_data(path=peng_dir["res16"] + "/train_triplets.txt",
                                     target_format="aos"),
        val = data_utils.read_data(path=peng_dir["res16"] + "/dev_triplets.txt",
                                     target_format="aos"),
        test = data_utils.read_data(path=peng_dir["res16"] + "/test_triplets.txt",
                                     target_format="aos")
    )
)

wan = dict(
    res15 = dict(
        train = data_utils.read_data(path=wan_dir["res15"] + "/train.txt",
                                     target_format="acs"),
        val = data_utils.read_data(path=wan_dir["res15"] + "/dev.txt",
                                     target_format="acs"),
        test = data_utils.read_data(path=wan_dir["res15"] + "/test.txt",
                                     target_format="acs")
    ),
    res16 = dict(
        train = data_utils.read_data(path=wan_dir["res16"] + "/train.txt",
                                     target_format="acs"),
        val = data_utils.read_data(path=wan_dir["res16"] + "/dev.txt",
                                     target_format="acs"),
        test = data_utils.read_data(path=wan_dir["res16"] + "/test.txt",
                                     target_format="acs")
    )
)

zhang = dict(
    res15 = dict(
        train = data_utils.read_data(path=zhang_dir["res15"] + "/train.txt",
                                     target_format="acso"),
        val = data_utils.read_data(path=zhang_dir["res15"] + "/dev.txt",
                                     target_format="acso"),
        test = data_utils.read_data(path=zhang_dir["res15"] + "/test.txt",
                                     target_format="acso")
    ),
    res16 = dict(
        train = data_utils.read_data(path=zhang_dir["res16"] + "/train.txt",
                                     target_format="acso"),
        val = data_utils.read_data(path=zhang_dir["res16"] + "/dev.txt",
                                     target_format="acso"),
        test = data_utils.read_data(path=zhang_dir["res16"] + "/test.txt",
                                     target_format="acso")
    )
)

william = dict(
    hotel = dict(
        train = data_utils.read_data(path=william_dir["hotel"] + "/train.txt",
                                     target_format="aos"),
        val = data_utils.read_data(path=william_dir["hotel"] + "/dev.txt",
                                     target_format="aos"),
        test = data_utils.read_data(path=william_dir["hotel"] + "/test.txt",
                                     target_format="aos")
    )
)

# Data Preprocessing 1

In [5]:
data_utils.SENTIMENT_ELEMENT = {'a' : "aspect", 'o' : "opinion", 's' : "sentiment", 'c' : "category"}

1. AOS (ASTE)
    * AO
    * AS
    * A
    * O

2. ACS (TASD)
    * AS
    * CS
    * A
    * C

3. ACOS
    * AO
    * AS
    * CS
    * A
    * O
    * C

In [6]:
task_tree = {
    "oas" : ["oas","oa","as",'a','o'],
    "asc" : ["asc","as","sc",'a','c'],
    "oasc" : ["oasc","oa","as","sc",'a','o','c']
}

all_task = []
for k,v1 in task_tree.items():
    if k not in all_task:
        all_task.append(k)
    for v2 in v1:
        if v2 not in all_task:
            all_task.append(v2)

print(all_task)

['oas', 'oa', 'as', 'a', 'o', 'asc', 'sc', 'c', 'oasc']


In [7]:
data_utils.remove_duplicate_targets(data_utils.reduce_targets([{'aspect': 'battery life', 'opinion': 'good', "sentiment" : "positive"},{'aspect': 'battery life', 'opinion': 'good', "sentiment" : "negative"}],"ao"))

[{'aspect': 'battery life', 'opinion': 'good'}]

Handle mix may not be a must, but we'll see it later. Will be problematic if like as (UABSA / E2E ABSA) used for training AOS (ASTE) --> may be for further experiment because we will insert imputing later on

In [8]:
data_utils.handle_mix_sentiment(data_utils.reduce_targets([{'aspect': 'battery life', 'opinion': 'good', "sentiment" : "positive"},{'aspect': 'battery life', 'opinion': 'good', "sentiment" : "negative"}],"aos"))

[{'aspect': 'battery life', 'opinion': 'good', 'sentiment': 'mixed'}]

In [9]:
from copy import deepcopy

# Peng (ASTE/AOS)
peng_intermediate = dict()

for domain, v1 in peng.items():
    peng_intermediate[domain] = dict()
    for task in ["oas"] + task_tree["oas"]:
        peng_intermediate[domain][task] = dict()
        for split in v1.keys():
            ds = peng[domain][split]
            ds_copy = deepcopy(ds)
            for i in range(len(ds_copy)):
                # Reduce
                ds_copy[i]["target"] = data_utils.reduce_targets(ds_copy[i]["target"],task)
                # Remove Duplicates
                ds_copy[i]["target"] = data_utils.remove_duplicate_targets(ds_copy[i]["target"])
            peng_intermediate[domain][task][split] = ds_copy

In [10]:
# Wan (TASD/ACS)
wan_intermediate = dict()

for domain, v1 in wan.items():
    wan_intermediate[domain] = dict()
    for task in ["asc"] + task_tree["asc"]:
        wan_intermediate[domain][task] = dict()
        for split in v1.keys():
            ds = wan[domain][split]
            ds_copy = deepcopy(ds)
            for i in range(len(ds_copy)):
                # Reduce
                ds_copy[i]["target"] = data_utils.reduce_targets(ds_copy[i]["target"],task)
                # Remove Duplicates
                ds_copy[i]["target"] = data_utils.remove_duplicate_targets(ds_copy[i]["target"])
            wan_intermediate[domain][task][split] = ds_copy

In [11]:
# Zhang (ACOS)
zhang_intermediate = dict()

for domain, v1 in zhang.items():
    zhang_intermediate[domain] = dict()
    for task in ["oasc"] + task_tree["oasc"]:
        zhang_intermediate[domain][task] = dict()
        for split in v1.keys():
            ds = zhang[domain][split]
            ds_copy = deepcopy(ds)
            for i in range(len(ds_copy)):
                # Reduce
                ds_copy[i]["target"] = data_utils.reduce_targets(ds_copy[i]["target"],task)
                # Remove Duplicates
                ds_copy[i]["target"] = data_utils.remove_duplicate_targets(ds_copy[i]["target"])
            zhang_intermediate[domain][task][split] = ds_copy

In [12]:
# William (AOS ID)
william_intermediate = dict()

for domain, v1 in william.items():
    william_intermediate[domain] = dict()
    for task in ["oas"] + task_tree["oas"]:
        william_intermediate[domain][task] = dict()
        for split in v1.keys():
            ds = william[domain][split]
            ds_copy = deepcopy(ds)
            for i in range(len(ds_copy)):
                # Reduce
                ds_copy[i]["target"] = data_utils.reduce_targets(ds_copy[i]["target"],task)
                # Remove Duplicates
                ds_copy[i]["target"] = data_utils.remove_duplicate_targets(ds_copy[i]["target"])
            william_intermediate[domain][task][split] = ds_copy

# Answer Engineering

In [13]:
mask = "<extra_id_X>"

In [14]:
added_tokens = {
    ',' : "<comma>",
    '(' : "<open_bracket>",
    ')' : "<close_bracket>",
    ';' : "<semicolon>"
}

In [15]:
# def construct_answer(targets,se_order):
#     result = []
#     counter = 0
#     for t in targets:
#         constructed_t = ""
#         for se in se_order:
#             if counter > 99:
#                 raise Exception("Extra id more than 99!")
#             constructed_t += ' ' + mask.replace('X',str(counter)) + ' ' + t[data_utils.SENTIMENT_ELEMENT[se]]
#             counter += 1
#         constructed_t = constructed_t.strip()
#         result.append(constructed_t)
#     result = " ; ".join(result)
#     return result
def construct_answer(targets,se_order):
    result = []
    for t in targets:
        constructed_t = []
        for se in se_order:
            element = t[data_utils.SENTIMENT_ELEMENT[se]]
            for k, v in added_tokens.items():
                element = element.replace(k,v)
            constructed_t.append(element)
        constructed_t = " , ".join(constructed_t)
        constructed_t = f"( {constructed_t} )"
        result.append(constructed_t)
    result = " ; ".join(result)
    return result

In [16]:
construct_answer(peng_intermediate["lap14"]["oas"]["train"][4]["target"],"oas")

'( no , GUI , negative ) ; ( dark , screen , negative ) ; ( steady , power light , neutral ) ; ( steady , hard drive light , negative )'

In [17]:
construct_answer([{"aspect" : "tes1 , tes2", "opinion" : "( tes3 ; tes4 )", "sentiment" : "positive"}],"oas")

'( <open_bracket> tes3 <semicolon> tes4 <close_bracket> , tes1 <comma> tes2 , positive )'

# Prompt Engineering

In [18]:
# def construct_prompt(text,se_order):
#     prompt = []
#     for counter, se in enumerate(se_order):
#         prompt.append(data_utils.SENTIMENT_ELEMENT[se] + " : " + mask.replace('X',str(counter)))
#     prompt = " ,".join(prompt)
#     result = text + "| " + prompt
#     return result
def construct_prompt(text,se_order):
    prompt = []
    for se in se_order:
        prompt.append(data_utils.SENTIMENT_ELEMENT[se])
    prompt = " , ".join(prompt)
    prompt = f"( {prompt} )"
    masked_text = text
    for k, v in added_tokens.items():
        masked_text = masked_text.replace(k,v)
    result = masked_text + " | " + prompt
    return result

In [19]:
construct_prompt(peng_intermediate["lap14"]["oas"]["train"][4]["text"],"oas")

'One night I turned the freaking thing off after using it <comma> the next day I turn it on <comma> no GUI <comma> screen all dark <comma> power light steady <comma> hard drive light steady and not flashing as it usually does . | ( opinion , aspect , sentiment )'

# Answer Catch

In [93]:
import re

# def catch_answer(output,se_order):
#     output = output.replace("<pad>",'')
#     output = output.replace("</s>",'')
#     pattern = r""
#     for se in se_order:
#         if se != 's':
#             pattern += f"<extra_id_\d+>\s*(?P<{data_utils.SENTIMENT_ELEMENT[se]}>[^;]+)\s*"
#         else:
#             pattern += f"<extra_id_\d+>\s*(?P<{data_utils.SENTIMENT_ELEMENT['s']}>positive|negative|neutral)\s*"
#     found = [found_iter.groupdict() for found_iter in re.finditer(pattern,output)]
#     for i in range(len(found)):
#         for k, v in found[i].items():
#             found[i][k] = found[i][k].strip()
#     return found
def catch_answer(output,se_order):
    # output = output.replace("<pad>",'')
    # output = output.replace("</s>",'')
    # output = output.split("<|sep|>")[-1]
    pattern = []
    for se in se_order:
        if se != 's':
            pattern.append(f"\s*(?P<{data_utils.SENTIMENT_ELEMENT[se]}>[^;]+)\s*")
        else:
            pattern.append(f"\s*(?P<{data_utils.SENTIMENT_ELEMENT['s']}>positive|negative|neutral)\s*")
    pattern = ','.join(pattern)
    pattern = f"\({pattern}\)"
    found = [found_iter.groupdict() for found_iter in re.finditer(pattern,output)]
    for i in range(len(found)):
        for k, v in found[i].items():
            found[i][k] = found[i][k].strip()
    return found

In [21]:
output = construct_answer(peng_intermediate["lap14"]["oas"]["train"][4]["target"],"oas")
se_order = "oas"
catch_answer(output,se_order)

[{'opinion': 'no', 'aspect': 'GUI', 'sentiment': 'negative'},
 {'opinion': 'dark', 'aspect': 'screen', 'sentiment': 'negative'},
 {'opinion': 'steady', 'aspect': 'power light', 'sentiment': 'neutral'},
 {'opinion': 'steady', 'aspect': 'hard drive light', 'sentiment': 'negative'}]

In [22]:
output

'( no , GUI , negative ) ; ( dark , screen , negative ) ; ( steady , power light , neutral ) ; ( steady , hard drive light , negative )'

# Data Preprocessing 2

In [23]:
from datasets import Dataset

peng_2 = dict()
for domain, v1 in peng_intermediate.items():
    peng_2[domain] = {
        "train" : [], # basic task
        "val" : [], # complex task
        "test" : [] # complex task
    }
    # TRAIN
    for basic_task in task_tree["oas"]:
        for el in peng_intermediate[domain][basic_task]["train"]:
            peng_2[domain]["train"].append({
                    "input" : construct_prompt(el["text"],basic_task),
                    "output" : construct_answer(el["target"],basic_task),
                    "task" : basic_task
                })
    # VAL
    for el in peng_intermediate[domain]["oas"]["val"]:
        peng_2[domain]["val"].append({
                "input" : construct_prompt(el["text"],"oas"),
                "output" : construct_answer(el["target"],"oas"),
                "task" : "oas"
            })
    # TEST
    for el in peng_intermediate[domain]["oas"]["test"]:
        peng_2[domain]["test"].append({
                "input" : construct_prompt(el["text"],"oas"),
                "output" : construct_answer(el["target"],"oas"),
                "task" : "oas"
            })
    peng_2[domain]["train"] = Dataset.from_list(peng_2[domain]["train"])
    peng_2[domain]["val"] = Dataset.from_list(peng_2[domain]["val"])
    peng_2[domain]["test"] = Dataset.from_list(peng_2[domain]["test"])

wan_2 = dict()
for domain, v1 in wan_intermediate.items():
    wan_2[domain] = {
        "train" : [], # basic task
        "val" : [], # complex task
        "test" : [] # complex task
    }
    # TRAIN
    for basic_task in task_tree["asc"]:
        for el in wan_intermediate[domain][basic_task]["train"]:
            wan_2[domain]["train"].append({
                    "input" : construct_prompt(el["text"],basic_task),
                    "output" : construct_answer(el["target"],basic_task),
                    "task" : basic_task
                })
    # VAL
    for el in wan_intermediate[domain]["asc"]["val"]:
        wan_2[domain]["val"].append({
                "input" : construct_prompt(el["text"],"asc"),
                "output" : construct_answer(el["target"],"asc"),
                "task" : "asc"
            })
    # TEST
    for el in wan_intermediate[domain]["asc"]["test"]:
        wan_2[domain]["test"].append({
                "input" : construct_prompt(el["text"],"asc"),
                "output" : construct_answer(el["target"],"asc"),
                "task" : "asc"
            })
    wan_2[domain]["train"] = Dataset.from_list(wan_2[domain]["train"])
    wan_2[domain]["val"] = Dataset.from_list(wan_2[domain]["val"])
    wan_2[domain]["test"] = Dataset.from_list(wan_2[domain]["test"])

zhang_2 = dict()
for domain, v1 in zhang_intermediate.items():
    zhang_2[domain] = {
        "train" : [], # basic task
        "val" : [], # complex task
        "test" : [] # complex task
    }
    # TRAIN
    for basic_task in task_tree["oasc"]:
        for el in zhang_intermediate[domain][basic_task]["train"]:
            zhang_2[domain]["train"].append({
                    "input" : construct_prompt(el["text"],basic_task),
                    "output" : construct_answer(el["target"],basic_task),
                    "task" : basic_task
                })
    # VAL
    for el in zhang_intermediate[domain]["oasc"]["val"]:
        zhang_2[domain]["val"].append({
                "input" : construct_prompt(el["text"],"oasc"),
                "output" : construct_answer(el["target"],"oasc"),
                "task" : "oasc"
            })
    # TEST
    for el in zhang_intermediate[domain]["oasc"]["test"]:
        zhang_2[domain]["test"].append({
                "input" : construct_prompt(el["text"],"oasc"),
                "output" : construct_answer(el["target"],"oasc"),
                "task" : "oasc"
            })
    zhang_2[domain]["train"] = Dataset.from_list(zhang_2[domain]["train"])
    zhang_2[domain]["val"] = Dataset.from_list(zhang_2[domain]["val"])
    zhang_2[domain]["test"] = Dataset.from_list(zhang_2[domain]["test"])

william_2 = dict()
for domain, v1 in william_intermediate.items():
    william_2[domain] = {
        "train" : [], # basic task
        "val" : [], # complex task
        "test" : [] # complex task
    }
    # TRAIN
    for basic_task in task_tree["oas"]:
        for el in william_intermediate[domain][basic_task]["train"]:
            william_2[domain]["train"].append({
                    "input" : construct_prompt(el["text"],basic_task),
                    "output" : construct_answer(el["target"],basic_task),
                    "task" : basic_task
                })
    # VAL
    for el in william_intermediate[domain]["oas"]["val"]:
        william_2[domain]["val"].append({
                "input" : construct_prompt(el["text"],"oas"),
                "output" : construct_answer(el["target"],"oas"),
                "task" : "oas"
            })
    # TEST
    for el in william_intermediate[domain]["oas"]["test"]:
        william_2[domain]["test"].append({
                "input" : construct_prompt(el["text"],"oas"),
                "output" : construct_answer(el["target"],"oas"),
                "task" : "oas"
            })
    william_2[domain]["train"] = Dataset.from_list(william_2[domain]["train"])
    william_2[domain]["val"] = Dataset.from_list(william_2[domain]["val"])
    william_2[domain]["test"] = Dataset.from_list(william_2[domain]["test"])

In [24]:
william_2["hotel"]["train"][69]

{'input': 'tempat yag bagus dan nyaman untuk istirahat tetapi tolong tvnya perlu di perbaiki channelnya karena banyak semutnya digambar dan water heaternya tidak bisa jadi mandi air dingin terus . | ( opinion , aspect , sentiment )',
 'output': '( bagus , tempat , positive ) ; ( nyaman , tempat , positive ) ; ( perlu di perbaiki , tvnya , positive ) ; ( tidak bisa , water heaternya , negative )',
 'task': 'oas'}

# Prepare Tokenized Dataset

## English

In [25]:
tokenizer_en = AutoTokenizer.from_pretrained("gpt2-large", padding_side="right")

In [26]:
encoding_args = {
    "max_length" : 128,
    "padding" : True,
    "truncation" : True,
    # "return_tensors" : "pt"
}

In [27]:
resize_en = False
if tokenizer_en.pad_token == None:
    pad_token = "<|pad|>"
    tokenizer_en.add_tokens([pad_token])
    tokenizer_en.add_special_tokens({"pad_token": pad_token})
    resize_en = True

if tokenizer_en.sep_token == None:
    sep_token = "<|sep|>"
    tokenizer_en.add_tokens([sep_token])
    tokenizer_en.add_special_tokens({"sep_token": sep_token})
    resize_en = True

Using pad_token, but it is not set yet.
Using sep_token, but it is not set yet.


In [28]:
def encode_en(dataset):
    causal_lm_input =[row_input + ' ' + tokenizer_en.sep_token + ' ' + row_output + ' ' + tokenizer_en.eos_token
                      for row_input, row_output in zip(dataset["input"],dataset["output"])]
    result = tokenizer_en(causal_lm_input, **encoding_args)
    return result

In [29]:
peng_tok = dict()
for domain, v1 in peng_2.items():
    peng_tok[domain] = dict()
    for split, v2 in v1.items():
        if split != "test":
            peng_tok[domain][split] = peng_2[domain][split].map(encode_en,batched=True,remove_columns=["input","output","task"])
        else:
            test_input = [row_input + ' ' + tokenizer_en.sep_token for row_input in peng_2[domain][split]["input"]]
            peng_tok[domain][split] = tokenizer_en(test_input, max_length=86, padding=True, truncation=True)

In [30]:
wan_tok = dict()
for domain, v1 in wan_2.items():
    wan_tok[domain] = dict()
    for split, v2 in v1.items():
        if split != "test":
            wan_tok[domain][split] = wan_2[domain][split].map(encode_en,batched=True,remove_columns=["input","output","task"])
        else:
            test_input = [row_input + ' ' + tokenizer_en.sep_token for row_input in wan_2[domain][split]["input"]]
            wan_tok[domain][split] = tokenizer_en(test_input, max_length=86, padding=True, truncation=True)

In [31]:
zhang_tok = dict()
for domain, v1 in zhang_2.items():
    zhang_tok[domain] = dict()
    for split, v2 in v1.items():
        if split != "test":
            zhang_tok[domain][split] = zhang_2[domain][split].map(encode_en,batched=True,remove_columns=["input","output","task"])
        else:
            test_input = [row_input + ' ' + tokenizer_en.sep_token for row_input in zhang_2[domain][split]["input"]]
            zhang_tok[domain][split] = tokenizer_en(test_input, max_length=86, padding=True, truncation=True)

## Indo

In [32]:
tokenizer_id = AutoTokenizer.from_pretrained("facebook/xglm-564M", padding_side="right")

In [33]:
resize_id = False
if tokenizer_id.pad_token == None:
    pad_token = "<|pad|>"
    tokenizer_id.add_tokens([pad_token])
    tokenizer_id.add_special_tokens({"pad_token": pad_token})
    resize_id = True

if tokenizer_id.sep_token == None:
    sep_token = "<|sep|>"
    tokenizer_id.add_tokens([sep_token])
    tokenizer_id.add_special_tokens({"sep_token": sep_token})
    resize_id = True

In [34]:
def encode_id(dataset):
    causal_lm_input =[row_input + ' ' + tokenizer_id.sep_token + ' ' + row_output + ' ' + tokenizer_id.eos_token
                      for row_input, row_output in zip(dataset["input"],dataset["output"])]
    result = tokenizer_id(causal_lm_input, **encoding_args)
    return result

In [35]:
william_tok = dict()
for domain, v1 in william_2.items():
    william_tok[domain] = dict()
    for split, v2 in v1.items():
        if split != "test":
            william_tok[domain][split] = william_2[domain][split].map(encode_id,batched=True,remove_columns=["input","output","task"])
        else:
            test_input = [row_input + ' ' + tokenizer_id.sep_token for row_input in william_2[domain][split]["input"]]
            william_tok[domain][split] = tokenizer_id(test_input, max_length=86, padding=True, truncation=True, return_tensors="pt")

# Data Collator

## English

In [36]:
from transformers import DataCollatorForLanguageModeling

data_collator_en = DataCollatorForLanguageModeling(tokenizer=tokenizer_en,mlm=False)

## Indo

In [37]:
data_collator_id = DataCollatorForLanguageModeling(tokenizer=tokenizer_id,mlm=False)

# Compute Metrics

In [38]:
from transformers import EvalPrediction
from evaluation import recall, precision, f1_score, summary_score
from typing import List, Dict, Tuple
import numpy as np

def seperate_target_prediction_per_task(predictions:List[List[Dict]],targets:List[List[Dict]],tasks:List) -> Tuple[Dict[str,List],Dict[str,List]]:
    per_task_targets = {}
    per_task_predictions = {}
    for target, prediction, task in zip(targets,predictions,tasks):
        if task not in per_task_targets.keys():
            per_task_targets[task] = []
        if task not in per_task_predictions.keys():
            per_task_predictions[task] = []
        per_task_targets[task].append(target)
        per_task_predictions[task].append(prediction)
    return per_task_targets, per_task_predictions

def preprocess_eval_preds(eval_preds:EvalPrediction,decoding_args:Dict[str,str],tokenizer:AutoTokenizer):
    input_ids = eval_preds.inputs
    target_ids = eval_preds.label_ids
    pred_ids = eval_preds.predictions

    # In case the model returns more than the prediction logits
    if isinstance(input_ids, tuple):
        input_ids = input_ids[0]
    if isinstance(target_ids, tuple):
        target_ids = target_ids[0]
    if isinstance(pred_ids, tuple):
        pred_ids = pred_ids[0]
    
    input_ids = np.argmax(input_ids,axis=-1) if len(input_ids.shape) == 3 else input_ids # in case not predict with generate
    target_ids = np.argmax(target_ids,axis=-1) if len(target_ids.shape) == 3 else target_ids # in case not predict with generate
    prediction_ids = np.argmax(pred_ids,axis=-1) if len(pred_ids.shape) == 3 else pred_ids # in case not predict with generate

    input_ids = [[token for token in row if token != -100] for row in input_ids]
    target_ids = [[token for token in row if token != -100] for row in target_ids]
    prediction_ids = [[token for token in row if token != -100] for row in prediction_ids]

    inputs = tokenizer.batch_decode(input_ids,**decoding_args)
    targets = tokenizer.batch_decode(target_ids,**decoding_args)
    predictions = tokenizer.batch_decode(prediction_ids,**decoding_args)

    return inputs, targets, predictions

def compute_metrics(eval_preds:EvalPrediction,decoding_args:Dict[str,str],tokenizer:AutoTokenizer,tasks:List) -> Dict[str,float]: # MAY NOT BE SUFFICIATE FOR CAUSAL LM
        """
        ### DESC
            Method to compute the metrics.
        ### PARAMS
        * eval_preds: EvalPrediction instance from training.
        * decoding_args: Decoding arguments.
        ### RETURN
        * metrics: Dictionary of metrics.
        """
        inputs, targets, predictions = preprocess_eval_preds(eval_preds,decoding_args,tokenizer)

        print(">> INPUTS :",inputs[0])
        print(">> TARGETS :",targets[0])
        print(">> PREDS :", predictions[0])

        targets = [catch_answer(text,task) for text,task in zip(targets,tasks) if task != "non_absa"]
        predictions = [catch_answer(text,task) for text,task in zip(predictions,tasks) if task != "non_absa"]


        per_task_targets, per_task_predictions = seperate_target_prediction_per_task(predictions, targets, tasks)
        
        metrics = {}

        metrics["overall_recall"] = recall(predictions,targets)
        metrics["overall_precision"] = precision(predictions,targets)
        metrics["overall_f1_score"] = f1_score(predictions,targets)

        for task in per_task_targets.keys():
            if task == "non_absa":
                continue
            metrics[f"{task}_recall"] = recall(per_task_predictions[task],per_task_targets[task])
            metrics[f"{task}_precision"] = precision(per_task_predictions[task],per_task_targets[task])
            metrics[f"{task}_f1_score"] = f1_score(per_task_predictions[task],per_task_targets[task])
        
        return metrics

# Train Arguments

In [39]:
from transformers import TrainingArguments

train_args = {
    "num_train_epochs": 10,
    "learning_rate": 3e-4,
    "save_total_limit": 2,
    "gradient_accumulation_steps": 2,
    "per_device_train_batch_size": 16//n_gpu,
    "per_device_eval_batch_size": 16//n_gpu,
    "save_strategy": "epoch",
    "evaluation_strategy": "epoch",
    "logging_strategy" : "epoch",
    # "metric_for_best_model": "overall_f1_score",
    # "load_best_model_at_end": True,
    "adam_epsilon": 1e-08,
    "output_dir": "./output",
    "logging_dir" : "./output/log",
    "include_inputs_for_metrics" : True
}

train_args = TrainingArguments(**train_args)

# Train

In [40]:
import torch
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda:0


In [41]:
from transformers import Trainer

# trainer = {
#     "peng" : {},
#     "wan" : {},
#     "zhang" : {},
#     "william" : {}
# }

decoding_args = {
    "skip_special_tokens" : False
}

def preprocess_logits_for_metrics(logits, targets):
    pred_logits = logits[0] if isinstance(logits,tuple) else logits
    pred_ids = torch.argmax(pred_logits, dim=-1)
    return pred_ids, targets

In [42]:
from tqdm import tqdm

def generate_predictions(model,tokenizer,data,device=torch.device("cuda:0"),decoding_args:Dict={}) -> List[str]:
    # Data loader
    # input_ids_data_loader = torch.utils.data.DataLoader(tokenized["input_ids"],
    #                     batch_size=batch_size,shuffle=False)
    # attention_mask_data_loader = torch.utils.data.DataLoader(tokenized["attention_mask"],
    #                     batch_size=batch_size,shuffle=False)
    # Predict
    model = model
    tokenizer = tokenizer
    tensor_predictions = []
    with torch.no_grad():
        for text in tqdm(data):
            # input_ids = input_ids.to(device)
            # attention_mask = attention_mask.to(device)
            input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)
            tensor_predictions.extend(model.generate(input_ids=input_ids, pad_token_id=tokenizer.pad_token_id,eos_token_id=tokenizer.eos_token_id,max_length=128).cpu())
            input_ids = input_ids.cpu()
            # attention_mask = attention_mask.cpu()
    tensor_predictions = [[token for token in row if token != -100] for row in tensor_predictions]
    predictions = tokenizer.batch_decode(tensor_predictions,**decoding_args)
    predictions = [el for el in predictions]
    return predictions

In [43]:
import json

def save_result(str_preds_,preds,targets,filename):
    result = []
    str_preds = [el.replace("<|pad|>",'').split("<|sep|>")[-1] for el in str_preds_]
    assert len(str_preds) == len(preds) == len(targets)
    for i in range(len(str_preds)):
        result.append({
            "str_pred" : str_preds[i],
            "pred" : preds[i],
            "target" : targets[i]
        })
    
    with open(filename,'w') as fp:
        json.dump(result,fp)
    return result

# Peng Laptop 2014

In [44]:
model = AutoModelForCausalLM.from_pretrained("gpt2-large")
if resize_en:
    model.resize_token_embeddings(len(tokenizer_en))
model.to(device)
trainer = Trainer(
        model = model,
        args = train_args,
        tokenizer = tokenizer_en,
        data_collator = data_collator_en,
        train_dataset = peng_tok["lap14"]["train"],
        eval_dataset = peng_tok["lap14"]["val"],
        compute_metrics = lambda eval_preds: compute_metrics(eval_preds,decoding_args,tokenizer_en,peng_2["lap14"]["val"]["task"]),
        preprocess_logits_for_metrics = preprocess_logits_for_metrics
    )

trainer.train()

/home/m13519061/anaconda3/envs/absa/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4530
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 1420
  Number of trainable parameters = 774032640
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Overall Recall,Overall Precision,Overall F1 Score,Oas Recall,Oas Precision,Oas F1 Score
1,1.059400,2.435000,0.296512,0.361404,0.325757,0.296512,0.361404,0.325757
2,0.234400,2.560602,0.322674,0.413919,0.362645,0.322674,0.413919,0.362645
3,0.171300,2.688858,0.360465,0.418605,0.387365,0.360465,0.418605,0.387365
4,0.146400,2.851537,0.348837,0.413793,0.378549,0.348837,0.413793,0.378549
5,0.134200,2.925708,0.340116,0.398649,0.367064,0.340116,0.398649,0.367064
6,0.130100,2.935990,0.348837,0.418118,0.380349,0.348837,0.418118,0.380349
7,0.125200,2.963400,0.363372,0.430034,0.393903,0.363372,0.430034,0.393903
8,0.122800,3.018411,0.372093,0.438776,0.402692,0.372093,0.438776,0.402692
9,0.120300,3.039836,0.360465,0.428082,0.391375,0.360465,0.428082,0.391375
10,0.117000,3.064008,0.372093,0.440273,0.403322,0.372093,0.440273,0.403322


***** Running Evaluation *****
  Num examples = 219
  Batch size = 16
  Num examples = 219
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-142
Configuration saved in ./output/checkpoint-142/config.json
Configuration saved in ./output/checkpoint-142/config.json
Model weights saved in ./output/checkpoint-142/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-142/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-142/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 219
  Batch size = 16
***** Running Evaluation *****
  Num examples = 219
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-284
Configuration saved in ./output/checkpoint-284/config.json
Configuration saved in ./output/checkpoint-284/config.json
Model weights saved in ./output/checkpoint-284/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-284/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-284/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 219
  Batch size = 16
***** Running Evaluation *****
  Num examples = 219
  Batch size = 16


...

Saving model checkpoint to ./output/checkpoint-994
Configuration saved in ./output/checkpoint-994/config.json
Configuration saved in ./output/checkpoint-994/config.json
Model weights saved in ./output/checkpoint-994/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-994/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-994/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-710] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 219
  Batch size = 16
***** Running Evaluation *****
  Num examples = 219
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-1136
Configuration saved in ./output/checkpoint-1136/config.json
Configuration saved in ./output/checkpoint-1136/config.json
Model weights saved in ./output/checkpoint-1136/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1136/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1136/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-852] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 219
  Batch size = 16
***** Running Evaluation *****
  Num examples = 219
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-1278
Configuration saved in ./output/checkpoint-1278/config.json
Configuration saved in ./output/checkpoint-1278/config.json
Model weights saved in ./output/checkpoint-1278/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1278/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1278/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-994] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 219
  Batch size = 16
***** Running Evaluation *****
  Num examples = 219
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-1420
Configuration saved in ./output/checkpoint-1420/config.json
Configuration saved in ./output/checkpoint-1420/config.json
Model weights saved in ./output/checkpoint-1420/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1420/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1420/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-1136] due to args.save_total_limit


TrainOutput(global_step=1420, training_loss=0.23611598887913662, metrics={'train_runtime': 2646.867, 'train_samples_per_second': 17.115, 'train_steps_per_second': 0.536, 'total_flos': 2.46447943406592e+16, 'train_loss': 0.23611598887913662, 'epoch': 10.0})

In [60]:
[el + ' ' + tokenizer_en.sep_token for el in peng_2["lap14"]["test"]["input"]][0]

'Boot time is super fast <comma> around anywhere from 35 seconds to 1 minute . | ( opinion , aspect , sentiment ) <|sep|>'

In [72]:
test_data = [el + ' ' + tokenizer_en.sep_token for el in peng_2["lap14"]["test"]["input"]]
str_preds = generate_predictions(model, tokenizer_en, test_data, device, decoding_args)
str_preds = [el.split(tokenizer_en.sep_token)[-1] for el in str_preds]
preds = [catch_answer(el,"oas") for el in str_preds]

100%|██████████| 328/328 [02:34<00:00,  2.13it/s]


In [76]:
targets = [catch_answer(el,"oas") for el in peng_2["lap14"]["test"]["output"]]

In [77]:
summary_score(preds,targets)

{'recall': 0.38817005545286504,
 'precision': 0.44421487603305787,
 'f1_score': 0.4143057053061412}

In [81]:
!rm -rf ./output

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [82]:
del model
torch.cuda.empty_cache()

In [83]:
result = save_result(str_preds, preds, targets, "peng_lap14.json")

# Peng Restaurant 2014

In [44]:
model = AutoModelForCausalLM.from_pretrained("gpt2-large")
if resize_en:
    model.resize_token_embeddings(len(tokenizer_en))
model.to(device)
trainer = Trainer(
        model = model,
        args = train_args,
        tokenizer = tokenizer_en,
        data_collator = data_collator_en,
        train_dataset = peng_tok["res14"]["train"],
        eval_dataset = peng_tok["res14"]["val"],
        compute_metrics = lambda eval_preds: compute_metrics(eval_preds,decoding_args,tokenizer_en,peng_2["res14"]["val"]["task"]),
        preprocess_logits_for_metrics = preprocess_logits_for_metrics
    )

trainer.train()

/home/m13519061/anaconda3/envs/absa/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6330
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 1980
  Number of trainable parameters = 774032640
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 310
  Batch size = 16
  Num examples = 310
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-198
Configuration saved in ./output/checkpoint-198/config.json
Configuration saved in ./output/checkpoint-198/config.json
Model weights saved in ./output/checkpoint-198/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-198/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-198/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 310
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-396
Configuration saved in ./output/checkpoint-396/config.json
Configuration saved in ./output/checkpoint-396/config.json
Model weights saved in ./output/checkpoint-396/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-396/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-396/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 310
  Batch size = 16


...

Saving model checkpoint to ./output/checkpoint-1386
Configuration saved in ./output/checkpoint-1386/config.json
Configuration saved in ./output/checkpoint-1386/config.json
Model weights saved in ./output/checkpoint-1386/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1386/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1386/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-990] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 310
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-1584
Configuration saved in ./output/checkpoint-1584/config.json
Configuration saved in ./output/checkpoint-1584/config.json
Model weights saved in ./output/checkpoint-1584/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1584/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1584/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-1188] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 310
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-1782
Configuration saved in ./output/checkpoint-1782/config.json
Configuration saved in ./output/checkpoint-1782/config.json
Model weights saved in ./output/checkpoint-1782/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1782/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1782/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-1386] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 310
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-1980
Configuration saved in ./output/checkpoint-1980/config.json
Configuration saved in ./output/checkpoint-1980/config.json
Model weights saved in ./output/checkpoint-1980/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1980/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1980/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-1584] due to args.save_total_limit


TrainOutput(global_step=1980, training_loss=0.23776443173186948, metrics={'train_runtime': 3658.1665, 'train_samples_per_second': 17.304, 'train_steps_per_second': 0.541, 'total_flos': 3.443797426176e+16, 'train_loss': 0.23776443173186948, 'epoch': 10.0})

In [45]:
test_data = [el + ' ' + tokenizer_en.sep_token for el in peng_2["res14"]["test"]["input"]]
str_preds = generate_predictions(model, tokenizer_en, test_data, device, decoding_args)
str_preds = [el.split(tokenizer_en.sep_token)[-1] for el in str_preds]
preds = [catch_answer(el,"oas") for el in str_preds]

100%|██████████| 492/492 [04:49<00:00,  1.70it/s]


In [46]:
targets = [catch_answer(el,"oas") for el in peng_2["res14"]["test"]["output"]]

In [47]:
summary_score(preds,targets)

{'recall': 0.5392354124748491,
 'precision': 0.5795698924731183,
 'f1_score': 0.558675595554636}

In [48]:
!rm -rf ./output

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [49]:
result = save_result(str_preds, preds, targets, "peng_res14.json")

# Peng Restaurant 2015

In [50]:
model = AutoModelForCausalLM.from_pretrained("gpt2-large")
if resize_en:
    model.resize_token_embeddings(len(tokenizer_en))
model.to(device)
trainer = Trainer(
        model = model,
        args = train_args,
        tokenizer = tokenizer_en,
        data_collator = data_collator_en,
        train_dataset = peng_tok["res15"]["train"],
        eval_dataset = peng_tok["res15"]["val"],
        compute_metrics = lambda eval_preds: compute_metrics(eval_preds,decoding_args,tokenizer_en,peng_2["res15"]["val"]["task"]),
        preprocess_logits_for_metrics = preprocess_logits_for_metrics
    )

trainer.train()

loading configuration file config.json from cache at /home/m13519061/.cache/huggingface/hub/models--gpt2-large/snapshots/212095d5832abbf9926672e1c1e8d14312a3be20/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2-large",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1280,
  "n_head": 20,
  "n_inner": null,
  "n_layer": 36,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  

Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 148
  Batch size = 16
  Num examples = 148
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-95
Configuration saved in ./output/checkpoint-95/config.json
Configuration saved in ./output/checkpoint-95/config.json
Model weights saved in ./output/checkpoint-95/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-95/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-95/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 148
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-190
Configuration saved in ./output/checkpoint-190/config.json
Configuration saved in ./output/checkpoint-190/config.json
Model weights saved in ./output/checkpoint-190/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-190/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-190/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 148
  Batch size = 16


...

Saving model checkpoint to ./output/checkpoint-665
Configuration saved in ./output/checkpoint-665/config.json
Configuration saved in ./output/checkpoint-665/config.json
Model weights saved in ./output/checkpoint-665/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-665/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-665/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-475] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 148
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-760
Configuration saved in ./output/checkpoint-760/config.json
Configuration saved in ./output/checkpoint-760/config.json
Model weights saved in ./output/checkpoint-760/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-760/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-760/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-570] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 148
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-855
Configuration saved in ./output/checkpoint-855/config.json
Configuration saved in ./output/checkpoint-855/config.json
Model weights saved in ./output/checkpoint-855/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-855/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-855/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-665] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 148
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-950
Configuration saved in ./output/checkpoint-950/config.json
Configuration saved in ./output/checkpoint-950/config.json
Model weights saved in ./output/checkpoint-950/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-950/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-950/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-760] due to args.save_total_limit


TrainOutput(global_step=950, training_loss=0.24644181100945722, metrics={'train_runtime': 1804.9995, 'train_samples_per_second': 16.759, 'train_steps_per_second': 0.526, 'total_flos': 1.64573257728e+16, 'train_loss': 0.24644181100945722, 'epoch': 10.0})

In [51]:
test_data = [el + ' ' + tokenizer_en.sep_token for el in peng_2["res15"]["test"]["input"]]
str_preds = generate_predictions(model, tokenizer_en, test_data, device, decoding_args)
str_preds = [el.split(tokenizer_en.sep_token)[-1] for el in str_preds]
preds = [catch_answer(el,"oas") for el in str_preds]

100%|██████████| 322/322 [02:47<00:00,  1.92it/s]


In [52]:
targets = [catch_answer(el,"oas") for el in peng_2["res15"]["test"]["output"]]

In [53]:
summary_score(preds,targets)

{'recall': 0.4556701030927835,
 'precision': 0.4324853228962818,
 'f1_score': 0.4437751004016064}

In [54]:
!rm -rf ./output

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [55]:
result = save_result(str_preds, preds, targets, "peng_res15.json")

# Peng Restaurant 2016

In [56]:
model = AutoModelForCausalLM.from_pretrained("gpt2-large")
if resize_en:
    model.resize_token_embeddings(len(tokenizer_en))
model.to(device)
trainer = Trainer(
        model = model,
        args = train_args,
        tokenizer = tokenizer_en,
        data_collator = data_collator_en,
        train_dataset = peng_tok["res16"]["train"],
        eval_dataset = peng_tok["res16"]["val"],
        compute_metrics = lambda eval_preds: compute_metrics(eval_preds,decoding_args,tokenizer_en,peng_2["res16"]["val"]["task"]),
        preprocess_logits_for_metrics = preprocess_logits_for_metrics
    )

trainer.train()

loading configuration file config.json from cache at /home/m13519061/.cache/huggingface/hub/models--gpt2-large/snapshots/212095d5832abbf9926672e1c1e8d14312a3be20/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2-large",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1280,
  "n_head": 20,
  "n_inner": null,
  "n_layer": 36,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  

Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 210
  Batch size = 16
  Num examples = 210
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-134
Configuration saved in ./output/checkpoint-134/config.json
Configuration saved in ./output/checkpoint-134/config.json
Model weights saved in ./output/checkpoint-134/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-134/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-134/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 210
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-268
Configuration saved in ./output/checkpoint-268/config.json
Configuration saved in ./output/checkpoint-268/config.json
Model weights saved in ./output/checkpoint-268/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-268/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-268/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 210
  Batch size = 16


...

Saving model checkpoint to ./output/checkpoint-938
Configuration saved in ./output/checkpoint-938/config.json
Configuration saved in ./output/checkpoint-938/config.json
Model weights saved in ./output/checkpoint-938/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-938/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-938/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-670] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 210
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-1072
Configuration saved in ./output/checkpoint-1072/config.json
Configuration saved in ./output/checkpoint-1072/config.json
Model weights saved in ./output/checkpoint-1072/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1072/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1072/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-804] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 210
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-1206
Configuration saved in ./output/checkpoint-1206/config.json
Configuration saved in ./output/checkpoint-1206/config.json
Model weights saved in ./output/checkpoint-1206/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1206/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1206/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-938] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 210
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-1340
Configuration saved in ./output/checkpoint-1340/config.json
Configuration saved in ./output/checkpoint-1340/config.json
Model weights saved in ./output/checkpoint-1340/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1340/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1340/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-1072] due to args.save_total_limit


TrainOutput(global_step=1340, training_loss=0.24749831726301963, metrics={'train_runtime': 2506.1685, 'train_samples_per_second': 17.098, 'train_steps_per_second': 0.535, 'total_flos': 2.331227799552e+16, 'train_loss': 0.24749831726301963, 'epoch': 10.0})

In [57]:
test_data = [el + ' ' + tokenizer_en.sep_token for el in peng_2["res16"]["test"]["input"]]
str_preds = generate_predictions(model, tokenizer_en, test_data, device, decoding_args)
str_preds = [el.split(tokenizer_en.sep_token)[-1] for el in str_preds]
preds = [catch_answer(el,"oas") for el in str_preds]

100%|██████████| 326/326 [02:43<00:00,  2.00it/s]


In [58]:
targets = [catch_answer(el,"oas") for el in peng_2["res16"]["test"]["output"]]

In [59]:
summary_score(preds,targets)

{'recall': 0.5603112840466926,
 'precision': 0.5722772277227722,
 'f1_score': 0.5662310450157491}

In [60]:
!rm -rf ./output

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [61]:
result = save_result(str_preds, preds, targets, "peng_res16.json")

# Wan Restaurant 2015

In [62]:
model = AutoModelForCausalLM.from_pretrained("gpt2-large")
if resize_en:
    model.resize_token_embeddings(len(tokenizer_en))
model.to(device)
trainer = Trainer(
        model = model,
        args = train_args,
        tokenizer = tokenizer_en,
        data_collator = data_collator_en,
        train_dataset = wan_tok["res15"]["train"],
        eval_dataset = wan_tok["res15"]["val"],
        compute_metrics = lambda eval_preds: compute_metrics(eval_preds,decoding_args,tokenizer_en,wan_2["res15"]["val"]["task"]),
        preprocess_logits_for_metrics = preprocess_logits_for_metrics
    )

trainer.train()

loading configuration file config.json from cache at /home/m13519061/.cache/huggingface/hub/models--gpt2-large/snapshots/212095d5832abbf9926672e1c1e8d14312a3be20/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2-large",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1280,
  "n_head": 20,
  "n_inner": null,
  "n_layer": 36,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  

Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 10
  Batch size = 16
  Num examples = 10
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-175
Configuration saved in ./output/checkpoint-175/config.json
Configuration saved in ./output/checkpoint-175/config.json
Model weights saved in ./output/checkpoint-175/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-175/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-175/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 10
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-350
Configuration saved in ./output/checkpoint-350/config.json
Model weights saved in ./output/checkpoint-350/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-350/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-350/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 10
  Batch size = 16


...

Saving model checkpoint to ./output/checkpoint-1225
Configuration saved in ./output/checkpoint-1225/config.json
Configuration saved in ./output/checkpoint-1225/config.json
Model weights saved in ./output/checkpoint-1225/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1225/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1225/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-875] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 10
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-1400
Configuration saved in ./output/checkpoint-1400/config.json
Model weights saved in ./output/checkpoint-1400/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1400/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1400/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-1050] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 10
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-1575
Configuration saved in ./output/checkpoint-1575/config.json
Configuration saved in ./output/checkpoint-1575/config.json
Model weights saved in ./output/checkpoint-1575/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1575/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1575/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-1225] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 10
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-1750
Configuration saved in ./output/checkpoint-1750/config.json
Configuration saved in ./output/checkpoint-1750/config.json
Model weights saved in ./output/checkpoint-1750/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1750/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1750/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-1400] due to args.save_total_limit


TrainOutput(global_step=1750, training_loss=0.2484765657697405, metrics={'train_runtime': 3198.6644, 'train_samples_per_second': 17.507, 'train_steps_per_second': 0.547, 'total_flos': 3.04664543232e+16, 'train_loss': 0.2484765657697405, 'epoch': 10.0})

In [63]:
test_data = [el + ' ' + tokenizer_en.sep_token for el in wan_2["res15"]["test"]["input"]]
str_preds = generate_predictions(model, tokenizer_en, test_data, device, decoding_args)
str_preds = [el.split(tokenizer_en.sep_token)[-1] for el in str_preds]
preds = [catch_answer(el,"asc") for el in str_preds]

100%|██████████| 582/582 [04:42<00:00,  2.06it/s]


In [64]:
targets = [catch_answer(el,"asc") for el in wan_2["res15"]["test"]["output"]]

In [65]:
summary_score(preds,targets)

{'recall': 0.463905325443787,
 'precision': 0.49936628643852976,
 'f1_score': 0.4809830929684312}

In [66]:
!rm -rf ./output

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [67]:
result = save_result(str_preds, preds, targets, "wan_res15.json")

# Wan Restaurant 2016

In [68]:
model = AutoModelForCausalLM.from_pretrained("gpt2-large")
if resize_en:
    model.resize_token_embeddings(len(tokenizer_en))
model.to(device)
trainer = Trainer(
        model = model,
        args = train_args,
        tokenizer = tokenizer_en,
        data_collator = data_collator_en,
        train_dataset = wan_tok["res16"]["train"],
        eval_dataset = wan_tok["res16"]["val"],
        compute_metrics = lambda eval_preds: compute_metrics(eval_preds,decoding_args,tokenizer_en,wan_2["res16"]["val"]["task"]),
        preprocess_logits_for_metrics = preprocess_logits_for_metrics
    )

trainer.train()

loading configuration file config.json from cache at /home/m13519061/.cache/huggingface/hub/models--gpt2-large/snapshots/212095d5832abbf9926672e1c1e8d14312a3be20/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2-large",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1280,
  "n_head": 20,
  "n_inner": null,
  "n_layer": 36,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  

Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 29
  Batch size = 16
  Num examples = 29
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-267
Configuration saved in ./output/checkpoint-267/config.json
Configuration saved in ./output/checkpoint-267/config.json
Model weights saved in ./output/checkpoint-267/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-267/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-267/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 29
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-534
Configuration saved in ./output/checkpoint-534/config.json
Configuration saved in ./output/checkpoint-534/config.json
Model weights saved in ./output/checkpoint-534/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-534/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-534/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 29
  Batch size = 16


...

Saving model checkpoint to ./output/checkpoint-1869
Configuration saved in ./output/checkpoint-1869/config.json
Configuration saved in ./output/checkpoint-1869/config.json
Model weights saved in ./output/checkpoint-1869/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1869/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1869/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-1335] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 29
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-2136
Configuration saved in ./output/checkpoint-2136/config.json
Configuration saved in ./output/checkpoint-2136/config.json
Model weights saved in ./output/checkpoint-2136/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-2136/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-2136/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-1602] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 29
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-2403
Configuration saved in ./output/checkpoint-2403/config.json
Configuration saved in ./output/checkpoint-2403/config.json
Model weights saved in ./output/checkpoint-2403/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-2403/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-2403/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-1869] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 29
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-2670
Configuration saved in ./output/checkpoint-2670/config.json
Configuration saved in ./output/checkpoint-2670/config.json
Model weights saved in ./output/checkpoint-2670/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-2670/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-2670/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-2136] due to args.save_total_limit


TrainOutput(global_step=2670, training_loss=0.25343111874012464, metrics={'train_runtime': 4829.9114, 'train_samples_per_second': 17.681, 'train_steps_per_second': 0.553, 'total_flos': 4.646134284288e+16, 'train_loss': 0.25343111874012464, 'epoch': 10.0})

In [69]:
test_data = [el + ' ' + tokenizer_en.sep_token for el in wan_2["res16"]["test"]["input"]]
str_preds = generate_predictions(model, tokenizer_en, test_data, device, decoding_args)
str_preds = [el.split(tokenizer_en.sep_token)[-1] for el in str_preds]
preds = [catch_answer(el,"asc") for el in str_preds]

100%|██████████| 587/587 [04:32<00:00,  2.16it/s]


In [70]:
targets = [catch_answer(el,"asc") for el in wan_2["res16"]["test"]["output"]]

In [71]:
summary_score(preds,targets)

{'recall': 0.5273573923166472,
 'precision': 0.568039950062422,
 'f1_score': 0.5469432053687286}

In [72]:
!rm -rf ./output

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [73]:
result = save_result(str_preds, preds, targets, "wan_res16.json")

# Zhang Restaurant 2015

In [74]:
model = AutoModelForCausalLM.from_pretrained("gpt2-large")
if resize_en:
    model.resize_token_embeddings(len(tokenizer_en))
model.to(device)
trainer = Trainer(
        model = model,
        args = train_args,
        tokenizer = tokenizer_en,
        data_collator = data_collator_en,
        train_dataset = zhang_tok["res15"]["train"],
        eval_dataset = zhang_tok["res15"]["val"],
        compute_metrics = lambda eval_preds: compute_metrics(eval_preds,decoding_args,tokenizer_en,zhang_2["res15"]["val"]["task"]),
        preprocess_logits_for_metrics = preprocess_logits_for_metrics
    )

trainer.train()

loading configuration file config.json from cache at /home/m13519061/.cache/huggingface/hub/models--gpt2-large/snapshots/212095d5832abbf9926672e1c1e8d14312a3be20/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2-large",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1280,
  "n_head": 20,
  "n_inner": null,
  "n_layer": 36,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  

Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 209
  Batch size = 16
  Num examples = 209
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-182
Configuration saved in ./output/checkpoint-182/config.json
Configuration saved in ./output/checkpoint-182/config.json
Model weights saved in ./output/checkpoint-182/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-182/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-182/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 209
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-364
Configuration saved in ./output/checkpoint-364/config.json
Configuration saved in ./output/checkpoint-364/config.json
Model weights saved in ./output/checkpoint-364/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-364/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-364/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 209
  Batch size = 16


...

Saving model checkpoint to ./output/checkpoint-1274
Configuration saved in ./output/checkpoint-1274/config.json
Configuration saved in ./output/checkpoint-1274/config.json
Model weights saved in ./output/checkpoint-1274/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1274/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1274/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-910] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 209
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-1456
Configuration saved in ./output/checkpoint-1456/config.json
Configuration saved in ./output/checkpoint-1456/config.json
Model weights saved in ./output/checkpoint-1456/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1456/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1456/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-1092] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 209
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-1638
Configuration saved in ./output/checkpoint-1638/config.json
Configuration saved in ./output/checkpoint-1638/config.json
Model weights saved in ./output/checkpoint-1638/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1638/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1638/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-1274] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 209
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-1820
Configuration saved in ./output/checkpoint-1820/config.json
Configuration saved in ./output/checkpoint-1820/config.json
Model weights saved in ./output/checkpoint-1820/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1820/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1820/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-1456] due to args.save_total_limit


TrainOutput(global_step=1820, training_loss=0.23226662206125784, metrics={'train_runtime': 3372.8296, 'train_samples_per_second': 17.309, 'train_steps_per_second': 0.54, 'total_flos': 3.17536620183552e+16, 'train_loss': 0.23226662206125784, 'epoch': 10.0})

In [75]:
test_data = [el + ' ' + tokenizer_en.sep_token for el in zhang_2["res15"]["test"]["input"]]
str_preds = generate_predictions(model, tokenizer_en, test_data, device, decoding_args)
str_preds = [el.split(tokenizer_en.sep_token)[-1] for el in str_preds]
preds = [catch_answer(el,"oasc") for el in str_preds]

100%|██████████| 537/537 [05:46<00:00,  1.55it/s]


In [76]:
targets = [catch_answer(el,"oasc") for el in zhang_2["res15"]["test"]["output"]]

In [77]:
summary_score(preds,targets)

{'recall': 0.3270440251572327,
 'precision': 0.33207070707070707,
 'f1_score': 0.3295381983349598}

In [78]:
!rm -rf ./output

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [79]:
result = save_result(str_preds, preds, targets, "zhang_res15.json")

# Zhang Restaurant 2016

In [80]:
model = AutoModelForCausalLM.from_pretrained("gpt2-large")
if resize_en:
    model.resize_token_embeddings(len(tokenizer_en))
model.to(device)
trainer = Trainer(
        model = model,
        args = train_args,
        tokenizer = tokenizer_en,
        data_collator = data_collator_en,
        train_dataset = zhang_tok["res16"]["train"],
        eval_dataset = zhang_tok["res16"]["val"],
        compute_metrics = lambda eval_preds: compute_metrics(eval_preds,decoding_args,tokenizer_en,zhang_2["res16"]["val"]["task"]),
        preprocess_logits_for_metrics = preprocess_logits_for_metrics
    )

trainer.train()

loading configuration file config.json from cache at /home/m13519061/.cache/huggingface/hub/models--gpt2-large/snapshots/212095d5832abbf9926672e1c1e8d14312a3be20/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2-large",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1280,
  "n_head": 20,
  "n_inner": null,
  "n_layer": 36,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  

Epoch,Training Loss,Validation Loss,Overall Recall,Overall Precision,Overall F1 Score,Oasc Recall,Oasc Precision,Oasc F1 Score
0,0.809900,1.841892,0.353175,0.410835,0.379829,0.353175,0.410835,0.379829
1,0.229800,2.062810,0.333333,0.361111,0.346667,0.333333,0.361111,0.346667
2,0.190800,2.154689,0.335317,0.381166,0.356775,0.335317,0.381166,0.356775
3,0.175900,2.152942,0.365079,0.412946,0.387540,0.365079,0.412946,0.387540


***** Running Evaluation *****
  Num examples = 316
  Batch size = 16
  Num examples = 316
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-276
Configuration saved in ./output/checkpoint-276/config.json
Configuration saved in ./output/checkpoint-276/config.json
Model weights saved in ./output/checkpoint-276/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-276/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-276/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 316
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-552
Configuration saved in ./output/checkpoint-552/config.json
Configuration saved in ./output/checkpoint-552/config.json
Model weights saved in ./output/checkpoint-552/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-552/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-552/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 316
  Batch size = 16


...

Saving model checkpoint to ./output/checkpoint-1932
Configuration saved in ./output/checkpoint-1932/config.json
Configuration saved in ./output/checkpoint-1932/config.json
Model weights saved in ./output/checkpoint-1932/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1932/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1932/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-1380] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 316
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-2208
Configuration saved in ./output/checkpoint-2208/config.json
Configuration saved in ./output/checkpoint-2208/config.json
Model weights saved in ./output/checkpoint-2208/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-2208/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-2208/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-1656] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 316
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-2484
Configuration saved in ./output/checkpoint-2484/config.json
Configuration saved in ./output/checkpoint-2484/config.json
Model weights saved in ./output/checkpoint-2484/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-2484/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-2484/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-1932] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 316
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-2760
Configuration saved in ./output/checkpoint-2760/config.json
Configuration saved in ./output/checkpoint-2760/config.json
Model weights saved in ./output/checkpoint-2760/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-2760/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-2760/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-2208] due to args.save_total_limit


TrainOutput(global_step=2760, training_loss=0.23229833409406137, metrics={'train_runtime': 5057.8658, 'train_samples_per_second': 17.494, 'train_steps_per_second': 0.546, 'total_flos': 4.81282931294208e+16, 'train_loss': 0.23229833409406137, 'epoch': 10.0})

In [81]:
test_data = [el + ' ' + tokenizer_en.sep_token for el in zhang_2["res16"]["test"]["input"]]
str_preds = generate_predictions(model, tokenizer_en, test_data, device, decoding_args)
str_preds = [el.split(tokenizer_en.sep_token)[-1] for el in str_preds]
preds = [catch_answer(el,"oasc") for el in str_preds]

100%|██████████| 544/544 [05:55<00:00,  1.53it/s]


In [82]:
targets = [catch_answer(el,"oasc") for el in zhang_2["res16"]["test"]["output"]]

In [83]:
summary_score(preds,targets)

{'recall': 0.3967459324155194,
 'precision': 0.3878048780487805,
 'f1_score': 0.39222445731894745}

In [84]:
!rm -rf ./output

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [85]:
result = save_result(str_preds, preds, targets, "zhang_res16.json")

# William Hotel

In [86]:
from transformers import AutoModelForCausalLM

In [87]:
model = AutoModelForCausalLM.from_pretrained("facebook/xglm-564M")
if resize_id:
    model.resize_token_embeddings(len(tokenizer_id))
model.to(device)
trainer = Trainer(
        model = model,
        args = train_args,
        tokenizer = tokenizer_id,
        data_collator = data_collator_en,
        train_dataset = william_tok["hotel"]["train"],
        eval_dataset = william_tok["hotel"]["val"],
        compute_metrics = lambda eval_preds: compute_metrics(eval_preds,decoding_args,tokenizer_id,william_2["hotel"]["val"]["task"]),
        preprocess_logits_for_metrics = preprocess_logits_for_metrics
    )

trainer.train()

loading configuration file config.json from cache at /home/m13519061/.cache/huggingface/hub/models--facebook--xglm-564M/snapshots/f3059f01b98ccc877c673149e0178c0e957660f9/config.json
Model config XGLMConfig {
  "_name_or_path": "facebook/xglm-564M",
  "activation_dropout": 0,
  "activation_function": "gelu",
  "architectures": [
    "XGLMForCausalLM"
  ],
  "attention_dropout": 0.1,
  "attention_heads": 16,
  "bos_token_id": 0,
  "d_model": 1024,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "eos_token_id": 2,
  "ffn_dim": 4096,
  "init_std": 0.02,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "xglm",
  "num_layers": 24,
  "pad_token_id": 1,
  "scale_embedding": true,
  "transformers_version": "4.24.0",
  "use_cache": true,
  "vocab_size": 256008
}

loading weights file pytorch_model.bin from cache at /home/m13519061/.cache/huggingface/hub/models--facebook--xglm-564M/snapshots/f3059f01b98ccc877c673149e0178c0e957660f9/pytorch_model.bin
All model checkpoint 

Epoch,Training Loss,Validation Loss,Overall Recall,Overall Precision,Overall F1 Score,Oas Recall,Oas Precision,Oas F1 Score
1,0.869600,0.662201,0.699697,0.721778,0.710566,0.699697,0.721778,0.710566


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
  Num examples = 1000
  Batch size = 16


>> INPUTS : </s> pelayanan sudah bagus <comma> tempat juga bagus <comma> akan lebih bagus lagi jika tempat untuk mencharger hp di tambah. terimakasih. | ( opinion, aspect, sentiment ) </s> ( bagus, pelayanan, positive ) ; ( bagus, tempat, positive ) </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
>> TARGETS : </s> pelayanan sudah bagus <comma> tempat juga bagus <comma> akan lebih bagus lagi jika tempat untuk mencharger hp di tambah. terimakasih. | ( opinion, aspect, sentiment ) </s> ( bagus, pelayanan, positive ) ; ( bagus, tempat, positive ) </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

Saving model checkpoint to ./output/checkpoint-469
Configuration saved in ./output/checkpoint-469/config.json
Configuration saved in ./output/checkpoint-469/config.json
Model weights saved in ./output/checkpoint-469/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-469/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-469/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16


...

Saving model checkpoint to ./output/checkpoint-3283
Configuration saved in ./output/checkpoint-3283/config.json
Configuration saved in ./output/checkpoint-3283/config.json
Model weights saved in ./output/checkpoint-3283/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-3283/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-3283/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-2345] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-3752
Configuration saved in ./output/checkpoint-3752/config.json
Configuration saved in ./output/checkpoint-3752/config.json
Model weights saved in ./output/checkpoint-3752/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-3752/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-3752/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-2814] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-4221
Configuration saved in ./output/checkpoint-4221/config.json
Configuration saved in ./output/checkpoint-4221/config.json
Model weights saved in ./output/checkpoint-4221/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-4221/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-4221/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-3283] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16


Saving model checkpoint to ./output/checkpoint-4690
Configuration saved in ./output/checkpoint-4690/config.json
Configuration saved in ./output/checkpoint-4690/config.json
Model weights saved in ./output/checkpoint-4690/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-4690/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-4690/special_tokens_map.json
Deleting older checkpoint [output/checkpoint-3752] due to args.save_total_limit


TrainOutput(global_step=4690, training_loss=0.18513852019808186, metrics={'train_runtime': 6293.9512, 'train_samples_per_second': 23.832, 'train_steps_per_second': 0.745, 'total_flos': 3.48262760448e+16, 'train_loss': 0.18513852019808186, 'epoch': 10.0})

In [88]:
test_data = [el + ' ' + tokenizer_id.sep_token for el in william_2["hotel"]["test"]["input"]]
str_preds = generate_predictions(model, tokenizer_id, test_data, device, decoding_args)
preds = [catch_answer(el,"oas") for el in str_preds]

100%|██████████| 1000/1000 [08:07<00:00,  2.05it/s]


In [102]:
str_preds = [el.split(tokenizer_id.sep_token)[2] for el in str_preds]
preds = [catch_answer(el,"oas") for el in str_preds]

In [89]:
targets = [catch_answer(el,"oas") for el in william_2["hotel"]["test"]["output"]]

In [103]:
summary_score(preds,targets)

{'recall': 0.6902916205241787,
 'precision': 0.7961783439490446,
 'f1_score': 0.7394636318341071}

In [91]:
!rm -rf ./output

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [104]:
result = save_result(str_preds, preds, targets, "william_hotel.json")

: 